**Chapter 7 – Ensemble Learning and Random Forests**

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥1.01.

In [16]:
# Python ≥3.7 is required
import sys
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥1.01 is required
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# INTRODUCTION

- It can be proven that if you aggregate (e.g. by taking the average prediction) the predictions of a group of predictors (such as classifiers or regressors), you will often get better predictions than with the best individual predictor.
- A group of predictors is called an _ensemble_.
- This technique is called _Ensemble Learning_.
- An Ensemble Learning algorithm is called an _Ensemble method_.

Example: 
- Train a group of Decision Tree classifiers, each on a different random subset of the training set. 
- To make predictions, you obtain the predictions of all the individual trees, then predict the class that gets the most votes. 
- Such an ensemble of Decision Trees is called a _Random Forest_.
- This is one of the most powerful Machine Learning algorithms available today.

# Voting Classifiers

Typically you choose several predictors to take part in the ensemble:

![](img/ensemble_diverse_predictors.png)

**Hard voting**

In _hard voting_ we predict the class that gets the most votes:

![](img/ensemble_hard_vorting.png)

Somewhat surprisingly, this voting classifier often achieves a higher accuracy than the best classifier in the ensemble.

Let's use the moons dataset to illustrate this and build a voting classifier:

In [17]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

When you fit a VotingClassifier, it clones every estimator and fits the clones. The
original estimators are available via the `estimators` attribute, while the fitted clones are available via the `estimators_` attribute. If you prefer a dict rather than a list, you can use `named_estimators` or `named_estimators_` instead.   

To begin, let’s look at each fitted classifier’s accuracy on the test set which is returned by de `score()` method of the classifier:

In [18]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, "=", clf.score(X_test, y_test))


lr = 0.864
rf = 0.896
svc = 0.896


When you call the voting classifier’s `predict()` method, it performs hard voting. For
example, the voting classifier predicts class 1 for the first instance of the test set, because two out of three classifiers predict that class:

In [19]:
voting_clf.predict(X_test[:1])

array([1])

In [20]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

Now let’s look at the performance of the voting classifier on the test set:

In [21]:
voting_clf.score(X_test, y_test)

0.912

There you have it! The voting classifier outperforms all the individual classifiers.

**Soft voting**
- If all classifiers are able to estimate class probabilities (i.e., they all have a `predict_proba()` method), then you can tell Scikit-Learn to predict the class with the
highest class probability, averaged over all the individual classifiers. 
- This is called `soft voting`. 
- It often achieves higher performance than hard voting because it gives more weight to highly confident votes.
- All you need to do is set the voting classifier’s voting hyperparameter to "soft", and ensure that all classifiers can estimate class probabilities. 
- This is not the case for the SVC class by default, so you need to set its probability hyperparameter to True (this will make the SVC class use cross-validation to estimate class probabilities, slowing down training, and it will add a `predict_proba()` method). 
- Let’s try that:

In [22]:
from sklearn.ensemble import VotingClassifier
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.92

We reach 92% accuracy simply by using soft voting—not bad!

# Bagging and Pasting
There are 2 ways to use a diverse set of classifiers:
1. use very different training algorithms (see above - Voting)
1. use the same training algorithm for every predictor and train them on different random subsets of the training set

**Bagging** and **pasting** is about this second way and refer to how this subset is chosen: 

- _Bagging_: sampling _with replacement_: each training sample can appear multiple times when training a single predictor.   
  &rarr; Imagine picking a card randomly from a deck of cards, writing it down, then placing it back in the deck
before picking the next card: the same card could be sampled multiple times.
- _Pasting_: sampling _without replacement_

(Bagging is short for _bootstrap aggregating_, which is the term used in statistics.)
  
Conclusion: 

- both bagging and pasting allow training instances to be sampled several times across multiple predictors.
- only bagging allows training instances to be sampled several times for the same predictor.

![](img/bagging-pasting.png) 
- The predictors in the above diagram are e.g. 4 Decision Tree Classfiers that are each trained with another subset of the training data.  
- Once all predictors are trained, the ensemble can make a prediction for a new instance by simply aggregating the predictions of all predictors.
  &rarr; Most frequent predictor (like hard voting) for classification and average for regression.
- Each individual predictor has a higher bias than if it were trained on the original training set (because it has fewer data). 
- Aggregation reduces both bias and variance.
- Generally, the net result is that the ensemble has a similar bias but a lower variance than a single predictor trained on the original training set.
- Predictors can all be trained in parallel, via different CPU cores or even different servers --> bagging and pasting scale very well. 


## Bagging and Pasting in Scikit-Learn
Use `BaggingClassifier` class or `BaggingRegressor` class

In [23]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1,random_state=42)  # n_jobs: number of CPU cores used for training and prediction (-1 = all available)
# our training data is still the moons dataset with 500 instances
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

**NOTE**  
The `BaggingClassifier` automatically performs soft voting instead of hard voting if the base
classifier can estimate class probabilities (i.e., if it has a `predict_proba()` method), which is the case
with Decision Tree classifiers.

In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.904


As compared to a single decison tree:

In [25]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.856


- The figure below compares the decision boundary of a single decision tree with the decision boundary of a bagging ensemble of 500 trees (from the preceding code), both trained on the moons dataset.   
- As we can see, the ensemble’s predictions will likely generalize much better (less overfitting!) than the single decision tree’s predictions: the ensemble has a comparable bias but a smaller variance (it makes roughly the same number of errors on the training set, but the decision boundary is less irregular).
![](img/dt_bagging.png)

## Out-of-Bag evaluation

- With bagging, some instances may be sampled several times for any given predictor, while others may not be sampled at all. 
- By default (parameter `max_samples` is omitted) a `BaggingClassifier` samples $m$ training instances with replacement (`bootstrap=True`), where $m$ is the size of the training set. 
- So in a training set of 100 samples we sample 100 times a single sample, but since each sample is replaced after sampling we have many duplicates. 
- With this process, it can be shown mathematically that only about 63% of the training instances are sampled on average for each predictor. 
- The remaining 37% of the training instances that are not sampled are called _out-of-bag_ (oob) instances. 
- Note that they are not the same 37% for all predictors.
- Since a predictor never sees the oob instances during training, it can be evaluated on these instances, without the need for a separate validation set. 
- Indeed, if there are enough estimators, then each instance in the training set will likely be an OOB instance of several estimators, so these estimators can be used to make a fair ensemble prediction for that instance.
- You can evaluate the ensemble itself by averaging out the oob evaluations of each predictor.
- The resulting evaluation score is available in the `oob_score_` attribute:

In [26]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=42)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

In [27]:
bag_clf.oob_decision_function_[:3]  # probas for the first 3 instances

array([[0.32352941, 0.67647059],
       [0.3375    , 0.6625    ],
       [1.        , 0.        ]])

Let's compare this with the accuracy on the test set:

In [28]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

We get 92% accuracy on the test. The OOB evaluation was a bit too pessimistic, just over 2% too low.

# Random Forests

- Random forest is an ensemble of decision trees, generally trained via the bagging method (or sometimes pasting) 
- Typically with `max_samples` set to the size of the training set. 
- Instead of building a `BaggingClassifier` and passing it a `DecisionTreeClassifier`, you can use the `RandomForestClassifier` class, which is more convenient and optimized for decision trees
- Similarly, there is a `RandomForestRegressor` class for regression tasks. 
- The following code trains a random forest classifier with 500 trees, each limited to maximum 16 leaf nodes, using all available CPU cores:

In [29]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

With a few exceptions, a `RandomForestClassifier` has all the hyperparameters of a `DecisionTreeClassifier` (to control how trees are grown), plus all the hyperparameters of a `BaggingClassifier` to control the ensemble itself.

- A Random Forest is equivalent to a bag of decision trees.
- By default, it samples $\sqrt{n}$ features (where n is the total number of features):

In [30]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, n_jobs=-1, random_state=42)

In [31]:
# extra code – verifies that the predictions are identical
bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)
np.all(y_pred_bag == y_pred_rf)  # same predictions

np.True_

## Feature Importance
Yet another great quality of Random Forests is that they make it easy to **measure the relative importance of each feature**.

In [32]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris.data, iris.target)
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
    print(round(score, 2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


In [33]:
rnd_clf.feature_importances_

array([0.11249225, 0.02311929, 0.44103046, 0.423358  ])

In [34]:
iris.data.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

The petal length and width are clearly much more important to determine the iris species than the sepal data. 

This figure shows the features (= pixel) importance for the MNIST dataset:  

![](img/mnist_pixel_importance.png)

Random forests are very handy to get a quick understanding of what features actually matter, in particular if you need to perform feature selection.

# Boosting
- Boosting refers to any Ensemble method that can combine several weak learners into a strong learner. 
- The general idea of most boosting methods is to train predictors sequentially, each trying to correct its predecessor. 
- There are many boosting methods available, but by far the most popular are _AdaBoost_ (short for _Adaptive Boosting_) and _Gradient Boosting_. 
- Let’s start with AdaBoost.
## AdaBoost
- One way for a new predictor to correct its predecessor is to pay a bit more attention to the training instances that the predecessor underfitted. 
- This results in new predictors are focusing more and more on the hard cases.
- The algorithm first trains a base classifier (such as a Decision Tree) and uses it to make predictions on the _training set_.
- The algorithm then increases the relative weight of misclassified training instances. 
- Then it trains a second classifier, using the updated weights, and again makes predictions on the training set, updates the instance weights, and so on;

![](img/adaboost.png)

- Scikit-Learn uses a multiclass version of AdaBoost called _SAMME_ (which stands for **Stagewise Additive Modeling using a Multiclass Exponential loss function**).
- The *learning rate* determines the weights of each classifier at each boosting iteration. 
- A *higher learning rate increases the contribution of each classifier*. There is a trade-off between the `learning_rate` and `n_estimators` parameters. Values must be in the range (0.0, inf).

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train) # we are still using the moons dataset

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

## Gradient Boosting

- Just like AdaBoost, Gradient Boosting works by sequentially adding predictors to an ensemble, each one correcting its predecessor. 
- However, instead of tweaking the instance weights at every iteration like AdaBoost does, this method tries to fit the new predictor to the residual
errors made by the previous predictor.

Let create a simple, noisy quadratic dataset and fit a `DecisionTreeRegressor` to it:

In [51]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)  # y = 3x² + Gaussian noise

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

Next, we’ll train a second DecisionTreeRegressor on the residual errors made by the first predictor:

In [39]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=43)

Then we train a third regressor on the residual errors made by the second predictor:

In [40]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=44)

Now we have an ensemble containing three trees. It can make predictions on a new instance simply by adding up the predictions of all the trees:

In [41]:
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

- The figure below represents the predictions of these three trees in the left column, and the ensemble’s predictions in the right column. 
- In the first row, the ensemble has just one tree, so its predictions are exactly the same as the first tree’s predictions. 
- In the second row, a new tree is trained on the residual errors of the first tree. 
- On the right you can see that the ensemble’s predictions are equal to the sum of the predictions of the first two trees. 
- Similarly, in the third row another tree is trained on the residual errors of the second tree. You can see that the ensemble’s predictions gradually get better as trees are added to the ensemble.
![](img/gradient_boosting_plot.png)

Now let's try a gradient boosting regressor. The following code creates the same ensemble as the previous one:

In [42]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3,
                                 learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

- The `learning_rate` hyperparameter scales the contribution of each tree. 
- If you set it to a low value, such as 0.1, you will need more trees in the ensemble to fit the training set, but the predictions will usually generalize better. 
- This is a regularization technique called `shrinkage`.

- The figure below shows two GBRT ensembles: 
    - the one on the left is based on the example above and does not have enough trees to fit the training set
    - the one on the right has too many trees and tends to overfit the training set.

![](img/gbrt_learning_rate_plot.png)

**Gradient Boosting with Early stopping**

- To find the optimal number of trees, you could perform cross-validation using GridSearchCV, as usual, but there’s a simpler way: 
    - if you set the `n_iter_no_change` hyperparameter to an integer value, say 10, then the Gradient
BoostingRegressor will automatically stop adding more trees during training if it sees that the last 10 trees didn’t help. 
- Let’s train the ensemble using early stopping:

In [47]:
gbrt_best = GradientBoostingRegressor(
    max_depth=2, learning_rate=0.05, n_estimators=500,
    n_iter_no_change=10, random_state=42)
gbrt_best.fit(X, y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

In [48]:
gbrt_best.n_estimators_

92

The validation errors are represented on the left of the figure, and the best model’s predictions are represented on the right (which should in our case be 92 iso 56).
![](img/early_stopping_gbrt_plot.png)

Several other optimized implementations of gradient boosting are
available in the Python ML ecosystem:
- **XGBoost** (very popular on Kaggle) 
- Cat‐Boost
- LightGBM
--------------------    
    
- They are all specialized for gradient boosting
- Their APIs are very similar to Scikit-Learn’s
- They provide many additional features, including GPU acceleration

# Stacking   
  
- The last ensemble method we will discuss in this chapter is called stacking (short for
stacked generalization).
- It is based on a simple idea: instead of using trivial functions (such as hard voting) to aggregate the predictions of all predictors in an ensemble, why don’t we train a model to perform this aggregation? 
- The figure below shows such an ensemble performing a regression task on a new instance. 
Each of the bottom three predictors predicts a different value (3.1, 2.7, and 2.9), and then the final predictor
(called a blender, or a meta learner) takes these predictions as inputs and makes the final prediction (3.0).

![](img/stacking1.png)

- To train the blender, you first need to build the blending training set. 
- You can use `cross_val_predict()` on every predictor in the ensemble to get predictions for each instance in the original training set and use these as the input features to train the blender
- The targets can simply be copied from the original training set. 
- Note that regardless of the number of features in the original training set (just one in this example), the blending training set will contain one input feature per predictor (three in this example). 

![](img/stacking2.png)


Scikit-Learn provides two classes for stacking ensembles: `StackingClassifier` and
`StackingRegressor`. 
 
For example, we can replace the `VotingClassifier` we used at
the beginning of this chapter on the moons dataset with a StackingClassifier:




In [52]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [53]:
stacking_clf.score(X_test, y_test)

0.928

If we evaluate this stacking model on the test set, we find 92.8% accuracy, which is a bit better than the voting classifier using soft voting, which got 92%.

> **Conclusion**:   
> Ensemble methods are versatile, powerful, and fairly simple to use.  
>  
> **Random forests, AdaBoost, and GBRT** are among **the first models you should test** for most machine learning tasks, and they particularly shine with heterogeneous tabular data. Moreover, as they require very little preprocessing, they’re great for getting a prototype up and running quickly.   
>
> Lastly, **ensemble methods** like voting classifiers and stacking classifiers **can help push your system’s performance** to its limits.